In [1]:
import os
%pwd

'c:\\Users\\PASCAL\\Chicken_Disease_Classification\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\PASCAL\\Chicken_Disease_Classification'

In [3]:
# Updating the entity (return type of a function) like configbox, and the example below

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

# same return from config.yaml

In [4]:
from chickenclassifier.constants import *
from chickenclassifier.utils.common import read_yaml, create_directories

In [5]:
# update congiuration manager cat configuration.py

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
    
        return data_ingestion_config

In [6]:
## Update components

import os
import urllib.request as request
import zipfile
from chickenclassifier.logger import logging
from chickenclassifier.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """Downloads the file and save into local directory
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logging.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """Extracts zipfile into data directory
            Returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
## Create the pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-27 00:34:49,904: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-27 00:34:49,909: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-27 00:34:49,911: INFO: common: created directory at: artifacts]
[2023-07-27 00:34:49,914: INFO: common: created directory at: artifacts/data_ingestion]


[2023-07-27 00:35:11,181: INFO: 947144846: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FD58:CC0E:6D366C:71EB4B:64C21E1B
Accept-Ranges: bytes
Date: Thu, 27 Jul 2023 07:34:52 GMT
Via: 1.1 varnish
X-Served-By: cache-ams21078-AMS
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1690443292.971310,VS0,VE756
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 4ccbf80b9209918d599c5b6cf2f52571d7c767c5
Expires: Thu, 27 Jul 2023 07:39:52 GMT
Source-Age: 0

]
